# Welcome

This notebook is a stream lined version of the original for those who are familiar with the settings

## Install Library

In [ ]:
%tensorflow_version 1.15.2
!nvidia-smi
!pip install -U miso2==2.4.4

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Thu Jan 13 11:31:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+------------------

## Optional: If training using folder on Google Drive

Mount google drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Optional: If uploading folder of images direct to Google Colab

Create datasets directory:

In [ ]:
import os
os.makedirs("/content/datasets/", exist_ok=True)

Upload to datasets then unzip:

In [ ]:
!unzip PASTE_HERE

# Training

In [ ]:
"""
Example image classification with the MISO library
"""
import ssl
from miso.training.parameters import MisoParameters
from miso.training.trainer import train_image_classification_model

# For downloading dataset over https
ssl._create_default_https_context = ssl._create_unverified_context


tp = MisoParameters()

# -----------------------------------------------------------------------------
# Name 
# -----------------------------------------------------------------------------
# A short name to describe the network
# The name will be also be used to construct the output directory name
# Leave as "" to auto-generate
tp.name = r"ResNet50 Cyclic TL"

# Description of this training run (leave as "" to auto-generate)
tp.description = "trained on google colab"

# -----------------------------------------------------------------------------
# Dataset
# -----------------------------------------------------------------------------
# Source directory (local folder or download link to dataset)
# Either a local directory, e.g. r'C:\Users\my_name\Documents\Data\particle_training_set\'
# or a direct download link, e.g. r"https://onedrive.live.com/download?cid=DAB7BF48C5EE0C24&resid=DAB7BF48C5EE0C24%21115338&authkey=AIoWuctsAijhpbQ"
tp.dataset.source = r"https://onedrive.live.com/download?cid=DAB7BF48C5EE0C24&resid=DAB7BF48C5EE0C24%21115338&authkey=AIoWuctsAijhpbQ"
# Minimum number of images to include in a class
tp.dataset.min_count = 10
# Whether to map the images in class with not enough examples to an "others" class
tp.dataset.map_others = False
# Fraction of dataset used for validation
tp.dataset.val_split = 0.2
# Random seed used to split the dataset into train and validation
tp.dataset.random_seed = 0
# Set to a local directory to stored the loaded dataset on disk instead of in memory
tp.dataset.memmap_directory = None

# -----------------------------------------------------------------------------
# CNN
# -----------------------------------------------------------------------------
# Choices for the CNN are:
# Network                   Description
# -----------------------   -----------------------------------------------------------------------
# base_cyclic               Classic sequential layer CNN with cyclic layers and batch normalisation
# resnet_cyclic             As above but with ResNet layers
# resnet50_tl               Transfer learning using ResNet50 (VERY FAST)
# resnet50_cyclic_tl        As above with added cyclic layers (VERY FAST)
# resnet50_cyclic_gain_tl   As above with added cyclic and gain layers (VERY FAST)
#
# (From the classifications_models library:)
# vgg16, vgg19 
# resnet18, resnet34, resnet50, resnet101, resnet152
# resnet50v2, resnet101v2, resnet152v2
# resnext50, resnext101, densenet121
# densenet169, densenet201
# inceptionv3, xception, inceptionresnetv2
# seresnet18, seresnet34, seresnet50, seresnet101, seresnet152
# seresnext50, seresnext101, senet154
# nasnetlarge, nasnetmobile
# mobilenet, mobilenetv2
tp.cnn.id = r"resnet50_cyclic_tl"

# The input dimensions of the image [height, width, channels]
# For height and width:
# - height and width should be the same for particle images to ensure proper rotation augmentation
# - 128 x 128 works well for simple particles
# - 224 x 224 is the size ResNet has been designed for
# - height and width will automatically be set when using transfer learning
# - NOTE: Larger images take longer to train!
# For number of channels:
# - using channels = 1 will convert the images to greyscale (recommended if colour is not a feature)
# - using channels = 3 will leave the image in RGB mode
tp.cnn.img_shape = [224, 224, 3]

# Input image colour space [greyscale/rgb]
tp.cnn.img_type = "rgb"

# For custom networks (base_cyclic / resnet_cyclic) or transfer learning
# Number of filters in first block (custom networks)
tp.cnn.filters = 4
# Number of blocks, set to None for automatic selection (custom networkss)
tp.cnn.blocks = None
# Size of dense layers (custom networks / transfer learning) as a list, e.g. [512, 512] for two dense layers size 512
# CURRENTLY FIXED AT SINGLE LAYER WITH DIMENSION 512
tp.cnn.dense = None
# Whether to use batch normalisation
tp.cnn.use_batch_norm = True
# Type of pooling [avg, max, none]
tp.cnn.global_pooling = "avg"
# Type of activation
tp.cnn.activation = "relu"
# Use A-Softmax (not implemented)
tp.cnn.use_asoftmax = False

# -----------------------------------------------------------------------------
# Training
# -----------------------------------------------------------------------------
# Number of images for each training step
# 64 is recommended if possible. Lower to 32 or 16 if getting out-of-memory errors
# Maximum value depends on GPU memory and image size
tp.training.batch_size = 32
# Number of epochs after which training is stopped regardless
# Keep at a high number like 10000 as training will normally
# be stopped by the adaptive learning rate system
tp.training.max_epochs = 10000
# Number of epochs and drops for the adaptive learning rate system. (ALR)
# ALR will monitor the last alr_epochs worth of epochs during training.
# If the loss is not decreasing, the learning rate will be dropped by half.
# After alr_drops times of drops, training is stopped.
# Recommend 10 for large datasets (>20000 images), 20 for medium (>2000), 40 for small
tp.training.alr_epochs = 10
# Number of learning rate drops after which training is suspended
tp.training.alr_drops = 4
# Default is to monitor the training loss, we can monitor the validation loss instead
tp.training.monitor_val_loss = False
# Use class weighting?
# Class weights are roughly the inverse of the frequency, clamped to the range [0.1, 10]
tp.training.use_class_weights = True
# Use class balancing via random over sampling? (Overrides class weights)
# This will balance by trying to use the same frequency of classes in each epoch
tp.training.use_class_undersampling = False
# Use train time augmentation?
# Augmente the images during training time (improves model performance significantly)
tp.training.use_augmentation = True

# -----------------------------------------------------------------------------
# Augmentation
# -----------------------------------------------------------------------------
# Setting depends on the size of list passed:
# - length 2, e.g. [low, high] = random value between low and high
# - length 3 or more, e.g. [a, b, c] = choose a random value from this list
# Rotation
tp.augmentation.rotation = [0, 360]
# Gain: I' = I * gain
tp.augmentation.gain = [0.8, 1.0, 1.2]
# Gamma: I' = I ^ gamma
tp.augmentation.gamma = [0.5, 1.0, 2.0]
# Bias: I' = I + bias
tp.augmentation.bias = None
# Zoom: I'[x,y] = I[x/zoom, y/zoom]
tp.augmentation.zoom = [0.9, 1.0, 1.1]
# Gaussian noise std deviation
tp.augmentation.gaussian_noise = None
# The parameters for the following are not random
# Random crop, e.g. [224, 224, 3]
# If random crop is used, you MUST set the original image size that the crop is taken from
tp.augmentation.random_crop = None
tp.augmentation.orig_img_shape = [224, 224, 3]

# -----------------------------------------------------------------------------
# Output
# -----------------------------------------------------------------------------
# Directory to save output (a sub-directory will be automatically created for each training run)
# Can be an absolute path, e.g. r'C:\Users\my_name\Documents\TrainedModels\'
# Or a relative path to the directory from which this script was run, e.g. r'output'
tp.output.save_dir = r"output"

# Save model?
tp.output.save_model = True
# Save the mislabelled image analysis?
# This can take some time...
tp.output.save_mislabeled = True


# model: Trained Keras model for image classification in inference mode
# vector_model: Sub-graph of previous model for generating feature vectors
# data_source: The training data
# results: Results of training (accuracy, precision, recall, f1score, etc)
model, vector_model, data_source, results = train_image_classification_model(tp)


+---------------------------------------------------------------------------+
| MISO Particle Classification Library                                      |
+---------------------------------------------------------------------------+
| Stable version:                                                           |
| pip install miso2                                                         |
| Development version:                                                      |
| pip install git+http://www.github.com/microfossil/particle-classification |
+---------------------------------------------------------------------------+
Tensorflow version: 1.15.2
--------------------------------------------------------------------------------
Train information:
- name: ResNet50 Cyclic TL
- description: trained on google colab
- CNN type: resnet50_cyclic_tl
- image type: rgb
- image shape: [224, 224, 3]

--------------------------------------------------------------------------------
Parsing source https://

100%|██████████| 27710/27710 [18:15<00:00, 25.30it/s]


--------------------------------------------------------------------------------
Transfer learning network training
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
94773248/94765736 [==============================] - 1s 0us/step
- calculating vectors
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.

! 749.3517587184906s elapsed, (27710/27710 vectors)
--------------------------------------------------------------------------------
Training
- class weights: [ 8.84870654  2.66126512  0.17082445  0.6981228   2.85442146  0.38264677
  0.16798684  0.55827802  1.29651378  0.1         0.1177081   4.65721397
  1.2332692   1.45060763  0.23013541  0.32472318  1.06610922  0.32652054
  2.04594371 10.          8.23135492  1.48717757  1.58012617 10.
  0.40497513  0.

Zip for download if desired:

In [ ]:
!zip -r /content/output.zip PASTE_PATH_HERE